In [5]:
import pandas as pd
# import utils.misc as misc
from config import *
# import yahoo_fin.stock_info as si
import os
from datetime import datetime, timedelta
from finvizfinance.screener.custom import Custom
from glob import glob

In [13]:
def add_ma(df, n):
    col_name = 'DMA'+str(n)
    df[col_name] = df["close"].rolling(n).mean()
    return df
def find_high(df):
    high = max(df['high'])
    high_date = (df[df['high'] == high].iloc[0,1])
    print(high, high_date)
    return (high, high_date)
def find_low(df):
    low = min(df['low'])
    low_date = df[df['low'] == low].iloc[0,1]
    # print(low, low_date)
    return (low, low_date)
def returns(buy_price, sell_price):
    return round((sell_price/buy_price - 1) * 100, 2)

In [7]:
def buy_hold(df, ex_date, next_ex_date = ''):
    output = dict()
    if len(next_ex_date) > 0:
        df_buy_hold = df[(df['date'] >= ex_date) & (df['date'] < next_ex_date)]
    else:
        df_buy_hold = df[df['date'] >= ex_date]
        df_buy_hold = df_buy_hold[:63]
    buy_price = df_buy_hold.iloc[0].open
    sell_price = df_buy_hold.iloc[-1].close
    output['bh_returns'] = returns(buy_price, sell_price)
    output['bh_close'] = sell_price
    output['bh_high'], x = find_high(df_buy_hold)
    output['bh_dth'] = df_buy_hold.loc[ex_date: x].shape[0] - 1
    output['bh_low'], y = find_low(df_buy_hold)
    output['bh_dtl'] = df_buy_hold.loc[ex_date: y].shape[0] - 1
    output['bh_high_returns'] = returns(buy_price, output['bh_high'])
    return output

In [8]:
def trail(df, SMA):
    sl = df.iloc[0].low
    exit_price = -1
    exit_method = ''

    dma_col = 'DMA'+str(SMA)
    df['above_DMA'] = (df['close'] >= df[dma_col])

    if df['above_DMA'].iloc[0] == False:
        print('Below DMA at entry, cannot trail!')
        return -1
    
    for i, row in df.iterrows():
        # print(i, row)
        if row.low < sl:
            exit_price = min(row.open, sl)
            exit_method = 'SL'
            break
        elif row.above_DMA == False:
            # print(row, dma_col)
            exit_price = min(row.open, row[dma_col])
            exit_method = 'DMA cross'
            break
        else:
            exit_price = row.close
            exit_method = 'In Position'
    return (exit_method, row.date, exit_price)

In [9]:
def trail_strategy(df, ex_date, SMA=10):
    output = dict()
    add_ma(df, SMA)
    df_trail = df[df['date'] >= ex_date]
    buy_price = df_trail.iloc[0].open
    temp = trail(df_trail,SMA)
    if temp == -1:
        return output
    else:
        exit_method, exit_date, sell_price = temp
    print(temp)
    df_trail = df_trail[df_trail['date'] <= exit_date]
    # print(df_trail)
    output['tr_returns'] = returns(buy_price, sell_price)
    output['tr_high'], x = find_high(df_trail)
    output['tr_dth'] = df_trail.loc[ex_date: x].shape[0] - 1
    output['tr_low'], y = find_low(df_trail)
    output['tr_dtl'] = df_trail.loc[ex_date: y].shape[0] - 1
    output['tr_high_returns'] = returns(buy_price, output['tr_high'])
    output['method'] = exit_method
    return output

In [14]:
if __name__ == "__main__":

    ticker = 'AAPL'
    df = pd.read_csv("ohlc/{}_ADJUSTED_OHLC.csv".format(ticker))
    df.drop(columns=['close'], axis=1, inplace=True)
    df.rename(columns={'adjclose': 'close'},inplace=True)
    df.index = df['date']
    
    #input
    ex_date = '2022-07-29'
    next_ex_date = '2022-10-28'     # if next_ex_date is not available, pass ""
    # next_ex_date = ""

    #holding until next earnings
    temp = buy_hold(df, ex_date, next_ex_date)
    print(temp)


    #trailing using 10 Day Moving Average
    ma_period = 10
    temp = trail_strategy(df, ex_date, ma_period)
    print(temp)

176.14999389648438 2022-08-17
{'bh_returns': -10.48, 'bh_close': 144.33981323242188, 'bh_high': 176.14999389648438, 'bh_dth': 13, 'bh_low': 134.3699951171875, 'bh_dtl': 53, 'bh_high_returns': 9.25}
('DMA cross', '2022-08-22', 169.69000244140625)
176.14999389648438 2022-08-17
{'tr_returns': 5.24, 'tr_high': 176.14999389648438, 'tr_dth': 13, 'tr_low': 159.5, 'tr_dtl': 0, 'tr_high_returns': 9.25, 'method': 'DMA cross'}


C:\Users\basil\AppData\Local\Temp\ipykernel_18504\2463640878.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['above_DMA'] = (df['close'] >= df[dma_col])


Qullamaggie trail method
1. Buy on ORB
2. Stops at LOD
3. Trail using 10 DMA